In [17]:
import urllib
import urllib.request
import json
import time
import numpy as np
from csv import writer
from datetime import datetime
import pandas as pd
import shutil

In [3]:
#urllib.request.urlopen('https://api.thingspeak.com/update?api_key=R7I8AQE9FRJBYGN9&field1='+str('adsfsf'))

In [4]:
#  a=urllib.request.urlopen('https://api.thingspeak.com/channels/508740/fields/2/last.json')
# print(a)


In [5]:
#  c= a.read()
#  print(c)

In [10]:
b = urllib.request.urlopen('https://api.thingspeak.com/channels/508740/fields/1/last.json').read()
res_dict = json.loads(b.decode('utf-8'))
# res_dict['field1']
res_dict

{'created_at': '2021-06-18T10:20:49Z',
 'entry_id': 683,
 'field1': '82670.05203_3_-0.209_8.736_4.685_5_-39.334_-8.984_11.604'}

In [11]:
mobile1_values = res_dict['field1']
type(mobile1_values)

str

In [17]:
a = urllib.request.urlopen('https://api.thingspeak.com/channels/508740/fields/2/last.json').read()
res_dict2 = json.loads(a.decode('utf-8'))
res_dict2

{'created_at': '2021-06-16T14:48:49Z',
 'entry_id': 669,
 'field2': '1457910.73129_3_-1.109_6.752_7.073'}

In [7]:
# mobile1_values = res_dict['feeds'][0]['field1']
# mobile1_values

In [8]:
# x=list(map(float, mobile1_values.split('_')))
# x

In [9]:
# i=0
# for item in x:  
#     i=i+1
#     if item==3.0:
#         acceleros=[x[i],x[i+1],x[i+2]]
#         sqrt_acceler = np.sqrt(sum(j*j for j in acceleros))
#         print(mobile1,"accel",sqrt_acceler)

#     if item==4.0:
#         gyros=[x[i],x[i+1],x[i+2]]
# #         sqrt_gyro = np.sqrt(sum(j*j for j in gyros))
#         print(mobile1,"Gyro:",sqrt_gyro)


In [6]:
head_cols = ['mobileno', 'value']

In [7]:
def get_data_mobile1():
    a = urllib.request.urlopen('https://api.thingspeak.com/channels/508740/fields/1/last.json').read()
    res_dict = json.loads(a.decode('utf-8'))
    
    mobile1_values = res_dict['field1']
    x = list(map(float, mobile1_values.split('_')))
    mobile = "mobile1"
    sqrt_acceler  = 0
    sqrt_gyro = 0
    i = 0
    for item in x:  
        i = i+1
        if item == 3.0:
            acceleros=[x[i],x[i+1],x[i+2]]
            sqrt_acceler = np.sqrt(sum(j*j for j in acceleros))
#             print(sqrt_acceler)

        if item == 4.0:
            gyros=[x[i],x[i+1],x[i+2]]
            sqrt_gyro = np.sqrt(sum(j*j for j in gyros))
#             print(sqrt_gyro)
            
#     return mobile, float(sqrt_acceler), float(sqrt_gyro)
    return mobile, mobile1_values

In [8]:
def get_data_mobile2():
    a = urllib.request.urlopen('https://api.thingspeak.com/channels/508740/fields/2/last.json').read()
    res_dict = json.loads(a.decode('utf-8'))

    mobile2_values = res_dict["field2"]
    x = list(map(float, mobile2_values.split('_')))
    i = 0
    mobile = "mobile2"
    
    sqrt_acceler  = 0
    sqrt_gyro = 0
    for item in x:
        i = i+1
        if item == 3.0:
            acceleros=[x[i],x[i+1],x[i+2]]
            sqrt_acceler = np.sqrt(sum(j*j for j in acceleros))
#             print(sqrt_acceler)

        if item == 4.0:
            gyros=[x[i],x[i+1],x[i+2]]
            sqrt_gyro = np.sqrt(sum(j*j for j in gyros))
#             print(sqrt_gyro)

    return mobile, mobile2_values

In [28]:
path = '/home/jovyan/work/mldb_proj/data//'

In [29]:
i=0
while True:
    time.sleep(5)
    i=i+1
    data1 = get_data_mobile1()
    data2 = get_data_mobile2()
    print("Data 1:",data1)
    print("Data 2:",data2)
#     with open('/home/jovyan/work/mldb_proj//data/file{}.csv'.format(i)) as f_object:
#             writer_object = writer(f_object)
# #             data = receive_data()
#             # Pass the list as an argument into
#             # the writerow()
#             writer_object.writerow(data1)
#             writer_object.writerow(data2)
#             #Close the file object
#             f_object.close()
    data =[data1, data2]
    df = pd.DataFrame(data,columns=head_cols)
    df.to_csv(path+'test{}.csv'.format(i), index =False)#-- for docker spark
  #  df.to_csv('C:/Users/asham/mlspark/mldb_proj//data/test{}.csv'.format(i), index =False) #for normal jupyter
    
#     print("Data added to data1test{}.csv".format(i))
#     print("Data added to data2test{}.csv".format(i))

Data 1: ('mobile1', '82670.05203_3_-0.209_8.736_4.685_5_-39.334_-8.984_11.604')
Data 2: ('mobile2', '1457910.73129_3_-1.109_6.752_7.073')
Data 1: ('mobile1', '82670.05203_3_-0.209_8.736_4.685_5_-39.334_-8.984_11.604')
Data 2: ('mobile2', '1457910.73129_3_-1.109_6.752_7.073')
Data 1: ('mobile1', '82670.05203_3_-0.209_8.736_4.685_5_-39.334_-8.984_11.604')
Data 2: ('mobile2', '1457910.73129_3_-1.109_6.752_7.073')
Data 1: ('mobile1', '82670.05203_3_-0.209_8.736_4.685_5_-39.334_-8.984_11.604')
Data 2: ('mobile2', '1457910.73129_3_-1.109_6.752_7.073')
Data 1: ('mobile1', '82670.05203_3_-0.209_8.736_4.685_5_-39.334_-8.984_11.604')
Data 2: ('mobile2', '1457910.73129_3_-1.109_6.752_7.073')
Data 1: ('mobile1', '82670.05203_3_-0.209_8.736_4.685_5_-39.334_-8.984_11.604')
Data 2: ('mobile2', '1457910.73129_3_-1.109_6.752_7.073')
Data 1: ('mobile1', '82670.05203_3_-0.209_8.736_4.685_5_-39.334_-8.984_11.604')
Data 2: ('mobile2', '1457910.73129_3_-1.109_6.752_7.073')
Data 1: ('mobile1', '82670.05203_3

KeyboardInterrupt: 

In [15]:
# def get_sqrt(mobileno, mobile_values):
#     sqrt_acceler = 0
#     sqrt_gyro = 0
    
#     x=list(map(float, mobile_values.split('_')))
#     i=0
#     for items in x:  
#         i=i+1
#         if items==3.0:
#             acceleros=[x[i],x[i+1],x[i+2]]
#             sqrt_acceler = np.sqrt(sum(j*j for j in acceleros))
# #             print(mobileno,"accel",sqrt_acceler)
#             i=0
# #             
#         if items==4.0:
#             gyros=[x[i],x[i+1],x[i+2]]
#             sqrt_gyro = np.sqrt(sum(j*j for j in gyros))
# #             print(mobileno,"Gyro:",sqrt_gyro)
#             i=0

#     return sqrt_acceler, sqrt_gyro 

In [16]:
# def read_data():
#     # Current date time in local system
#     dateTimeObj = datetime.now()
#     now = dateTimeObj.strftime("%d-%b-%Y (%H:%M:%S.%f)")
    
#     weight1 = 65
#     weight2 = 80
    
#     height1 = 170
#     height2 = 185
    
#     gender1 = 'M'
#     gender2 = 'M'
    
#     age1 = 25
#     age2 = 27
    
#     a=urllib.request.urlopen('https://api.thingspeak.com/channels/508740/feeds.json?results=1').read()
#     res_dict= json.loads(a.decode('utf-8'))
#     print("Receiving data from url")
#     mobile1 = res_dict['channel']['field1']
#     mobile1_values = res_dict['feeds'][0]['field1']
#     sqrt_acceler1, sqrt_gyro1 = get_sqrt(mobile1, mobile1_values)
# #     print("got sqrt of accelero:",sqrt_acceler1," and gyro:", sqrt_gyro1,"from ",mobile1)
    
#     sqrt_gyro2 = 0
#     sqrt_acceler2 = 0
#     mobile2 = res_dict['channel']['field2'] 
#     mobile2_values = res_dict['feeds'][0]['field2']
    
#     if mobile2_values is not None:
#         sqrt_gyro2, sqrt_acceler2 = get_sqrt(mobile2, mobile2_values)
# #         print("got sqrt of accelero:",sqrt_acceler2," and gyro:", sqrt_gyro2,"from ",mobile2)
#         return sqrt_gyro1, sqrt_acceler1, sqrt_gyro2, sqrt_acceler2
    
#     data1 = [mobile1, now, sqrt_acceler1, sqrt_gyro1, weight1, height1, gender1, age1]
#     data2 = [mobile2, now, sqrt_acceler2, sqrt_gyro2, weight2, height2, gender2, age2]
#     return data1, data2

In [20]:
i=0
while True:
#     time.sleep(2)
    i=i+1
    data1, data2 = read_data()
    print("Data 1:",data1)
    print("Data 2:",data2)
#     with open('/home/jovyan/work/mldb_proj//data/file{}.csv'.format(i)) as f_object:
#             writer_object = writer(f_object)
# #             data = receive_data()
#             # Pass the list as an argument into
#             # the writerow()
#             writer_object.writerow(data1)
#             writer_object.writerow(data2)
#             #Close the file object
#             f_object.close()
    data =[data1, data2]
    df = pd.DataFrame(data,columns=head_cols)
    df.to_csv(path+'test{}.csv'.format(i), index =False)
    
#     df = pd.DataFrame(data2, index=False)
#     df.to_csv('/home/jovyan/work/mldb_proj//data/data2test{}.csv'.format(i))

    
#     print("Data added to data1test{}.csv".format(i))
#     print("Data added to data2test{}.csv".format(i))

NameError: name 'read_data' is not defined